<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Сделаем-рассчет-результатов-для-наших-исходных-параметров" data-toc-modified-id="Сделаем-рассчет-результатов-для-наших-исходных-параметров-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Сделаем рассчет результатов для наших исходных параметров</a></span></li><li><span><a href="#Сделаем-рассчет-результатов-для-новых-параметров" data-toc-modified-id="Сделаем-рассчет-результатов-для-новых-параметров-3.0.2"><span class="toc-item-num">3.0.2&nbsp;&nbsp;</span>Сделаем рассчет результатов для новых параметров</a></span></li></ul></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Разница-получилась-минимальная,-значит-при-умножении-признаков-на-обратную-матрицу-качество-модели-не-изменилось." data-toc-modified-id="Разница-получилась-минимальная,-значит-при-умножении-признаков-на-обратную-матрицу-качество-модели-не-изменилось.-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Разница получилась минимальная, значит при умножении признаков на обратную матрицу качество модели не изменилось.</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
# Импортируем все необходимое
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

# Откроем файл и выведем первые 20 строк
try:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')
except:
    df = pd.read_csv('insurance.csv')
    
df.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


Посмотрим общую информацию по датафрейму

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Выведем общее количество явных дубликатов

In [3]:
df.duplicated().sum()

153

Избавимся от явных дубликатов

In [4]:
df.drop_duplicates(inplace=True)

Еще раз проверим количество дубликатов

In [5]:
df.duplicated().sum()

0

Проверим наличие пропущенных значений

In [6]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Изменим тип данных в некоторых колонках

In [7]:
df['Возраст'] = df['Возраст'].astype(int)
df['Члены семьи'] = df['Члены семьи'].astype(int)
df['Зарплата'] = df['Зарплата'].astype(int)

Вывод:

В данных нет критических нарушений. Были обнаружены явные дубликаты, но от них мы избавились, на исследование это не должно повлиять. Также для удобства мы изменили тип данных в некоторых колонках, где он был указан не совсем корректно. Учитывая цель нашего проекта данные манипуляции не являются обязательными, но для удобства мы все же выполнили их.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

<div class="alert alert-info">

    
$$
V = XP
$$
    
$$
A = Vw
$$    

$$
w = (X^T X)^{-1} X^T y
$$    
    
$$
W = (V^T V)^{-1} V^T y
$$
   
$$
W = ((XP)^T XP)^{-1} (XP)^T y
$$
    
$$
W = (P^T X^T XP)^{-1} P^T X^T y
$$

$$
W = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y  
$$    
    
Произведение $(P^T)^{-1} P^T$ = $E$ (единичная матрица) 
    
$$
W = P^{-1} (X^T X)^{-1} E X^T y 
$$    
    
$$
W = P^{-1} (X^T X)^{-1} X^T y 
$$

$$
W = P^{-1}w
$$

Подставим полученные значения в формулу $A = V w$:
    
$$
A = XP P^{-1}w
$$    

По условию $P$ обратимая, значит произведение $ P P^{-1}$ = $E$   
$$   
A = X E w    
$$    
    
$$    
A = Xw    
$$    


## Алгоритм преобразования

**Алгоритм**

1. Создание случайной матрицы
2. Проверка созданной матрицы на обратимость
3. Умножение признаков
4. Получить метрики обученной модели на исходных прзнаках и на умноженных признаках
5. Сравним метрики. Если они будут равны, то  мы достигли нужного результата

**Обоснование**

Выделим признаки и целевой признак

In [8]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

Создаем рандомную матрицу

In [9]:
r_matrix = np.random.rand(4, 4)

Проверяем существует ли обратная матрица для созданной нами

In [10]:
try:
    inv_matrix = np.linalg.inv(r_matrix)
    print('Обратная матрица существует')
except:
    print('Обратная матрица не существует')

Обратная матрица существует


#### Сделаем рассчет результатов для наших исходных параметров

$$
a = Xw
$$

$$
w = (X^T X)^{-1} X^T y
$$

In [11]:
w = np.linalg.inv(features.T.dot(features)).dot(features.T).dot(target)
a = features @ w

#### Сделаем рассчет результатов для новых параметров

In [12]:
new_features = features @ r_matrix

n_w = np.linalg.inv(new_features.T.dot(new_features)).dot(new_features.T).dot(target)
n_a = new_features @ n_w

In [13]:
dif = a - n_a

In [14]:
dif.sum()

0.02599669012652526

In [25]:
#КОД РЕВЬЮЕРА
np.allclose(a, n_a, atol=1e-4)

True

### Вывод:

Получившийся алгоритм не создает различий при использовании исходных и новых параметров, следовательно качество предсказаний модели не будет ухудшаться.

## Проверка алгоритма

Создадим класс модели линейной регрессии

In [15]:
class ConstantRegression:
    def model_fit(self, features, target):
        self.model = LinearRegression()
        self.model.fit(features, target)
        
    def predict_result(self, features):
        result = self.model.predict(features)
        return result
    
    def sm_r2_score(self, target, predict):
        r2 = r2_score(target, predict)
        return r2

Рассчитаем результат метрики для исходных признаков

In [16]:
model = ConstantRegression()
model.model_fit(features, target)
predictions_model = model.predict_result(features)
r2_model = model.sm_r2_score(target, predictions_model)

Рассчитаем результат метрики для признаков, умноженных на обратимую матрицу

In [17]:
new_model = ConstantRegression()
new_model.model_fit(new_features, target)
predictions_new_model = new_model.predict_result(new_features)
r2_new_model = new_model.sm_r2_score(target, predictions_new_model)

Сравним результаты

In [18]:
print('R2 на исходных признаках:', r2_model)
print('R2 при использовании умножения признаков на обратную матрицу:', r2_new_model)

R2 на исходных признаках: 0.4302010046633359
R2 при использовании умножения признаков на обратную матрицу: 0.4302010046627298


### Разница получилась минимальная, значит при умножении признаков на обратную матрицу качество модели не изменилось.

# Вывод

1. Критических нарушений, которые могли бы помешать исслудованию, мы не обнаружили. Присутствовали явные дубликаты, от них мы избавились. Также для удобства мы изменили тип данных в некоторых колонках, где он был указан не совсем корректно. Учитывая цель нашего проекта данные манипуляции не являются обязательными, но для удобства мы все же выполнили их.

2. Написали алгоритм для преобразования данных. Получившийся алгоритм не создает различий при использовании исходных и новых параметров, следовательно качество предсказаний модели не будет ухудшаться.

3. При проверке алгоритмы мы вяснили, что разница получается минимальная, а значит при умножении признаков на обратную матрицу качество модели не изменилось.

Качество работы линейной регрессии не изменилось, следовательно алгоритм преобразования данных работает правильно.
